In [ ]:
# !pip install -q git+https://github.com/Deci-AI/super-gradients.git@stable # 런타임 재시작
# !pip install -q supervision
# !pip install -q onemetric

In [1]:
import os
import shutil
from tqdm import tqdm
import random
import numpy as np
import pandas as pd
import torch
import yaml

In [2]:
PROJECT_PATH = '...'

In [3]:
MODEL_ARCH = 'yolo_nas_s' # 모델 아키텍처
BATCH_SIZE = 32
MAX_EPOCHS = 50
CHECKPOINT_DIR = PROJECT_PATH + 'nas_checkpoints'
EXPERIMENT_NAME = 'yolo_nas_s'

In [ ]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [5]:
dataset_params = {
    'data_dir': PROJECT_PATH,
    'train_images_dir':'sample_aug_train/images',
    'train_labels_dir':'sample_aug_train/labels',
    'val_images_dir':'sample_aug_val/images',
    'val_labels_dir':'sample_aug_val/labels',
    'classes': ['nong_a', 'bass', 'soong_a', 'gangjunchi', 'bluegill', 'ing_a', 'boong_a', 'noochi']
}

In [15]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

[2023-11-09 13:45:44] WARNING - yolo_format_detection.py - 4498 images are note associated to any label file
[2023-11-09 13:45:44] WARNING - yolo_format_detection.py - As a consequence, 14377/18875 images and 14377/14377 label files will be used.
[2023-11-09 13:45:44] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████████████████████████████████████████| 14377/14377 [00:01<00:00, 11102.02it/s]
[2023-11-09 13:45:45] WARNING - yolo_format_detection.py - 502 images are note associated to any label file
[2023-11-09 13:45:45] WARNING - yolo_format_detection.py - As a consequence, 1593/2095 images and 1593/1593 label files will be used.
[2023-11-09 13:45:45] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%

In [16]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)

model.to('cuda')

YoloNAS_S(
  (backbone): NStageBackbone(
    (stem): YoloNASStem(
      (conv): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        )
        (branch_1x1): Conv2d(3, 48, kernel_size=(1, 1), stride=(2, 2))
        (post_bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (rbr_reparam): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
    )
    (stage1): YoloNASStage(
      (downsample): QARepVGGBlock(
        (nonlinearity): ReLU(inplace=True)
        (se): Identity()
        (branch_3x3): Sequential(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=Tr

[2023-11-09 13:45:47] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-11-09 13:45:47] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


In [17]:
# nas 학습 파라미터 설정
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050, DetectionMetrics_075
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    #'resume': True,
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "AdamW",
    "optimizer_params": {"weight_decay": 0.001}, # 0.0001에서 0.001로 조정
    "zero_weight_decay_on_bias_and_bn": True, # 배치 정규화와 bias에 가중치 감쇠 적용 X
    "ema": True, # 지수 이동 평균
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False, # 정적할당
        num_classes=len(dataset_params['classes']),
        reg_max=16 # 바운딩 박스 예측과 실체 객체의 위치 간의 정확성을 조절하는 하이퍼파라미터
        # (예측된 위치가 실제 위치와 최대 16픽셀 떨어질 수 있도록 설정)
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.5, # confidence threshold 높여서 학습하자.
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.1, # 0.01에서 0.1로 조정
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [ ]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)